## This Notebook is intended to quickly run some sanity checks for the Aya 23 Models 
(I'll use the 8B and hope results extend to the larger one)

In [4]:
model_path = '/NS/llm-1/nobackup/afkhan/Model_Saves/aya-23-8B'

In [5]:
# Set CUDA visible devices to GPU 1
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [02:10<00:00, 32.57s/it]


In [8]:
device = 'cuda' 

model.to(device)

CohereForCausalLM(
  (model): CohereModel(
    (embed_tokens): Embedding(256000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x CohereDecoderLayer(
        (self_attn): CohereSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): CohereRotaryEmbedding()
        )
        (mlp): CohereMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): CohereLayerNorm()
      )
    )
    (norm): CohereLayerNorm()
  )
  (lm_head): Linear(in_features=409

In [10]:
!nvidia-smi

Sun Aug  4 07:50:03 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 PCIe               On  |   00000000:17:00.0 Off |                    0 |
| N/A   35C    P0             73W /  310W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

### Effect of Chat Template

In [17]:
# With Template

# Format message with the command-r-plus chat template
messages = [{"role": "user", "content": "गति कैसे परिभाषित की जाती है?"}]
input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")

## <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>गति कैसे परिभाषित की जाती है?<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>

input_ids = input_ids.to(device)

gen_tokens = model.generate(
    input_ids, 
    max_new_tokens=500, 
    do_sample=True, 
    temperature=0.3,
    )

gen_text = tokenizer.decode(gen_tokens[0])
print(gen_text)

<BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>गति कैसे परिभाषित की जाती है?<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>गति को आमतौर पर किसी वस्तु की स्थानिक बदलाव की दर के रूप में परिभाषित किया जाता है, जिसे आमतौर पर "दूरी के अनुसार समय" के रूप में व्यक्त किया जाता है। यह एक वस्तु के स्थान में परिवर्तन की दर को दर्शाता है, चाहे वह स्थिर हो या स्थानांतरित हो।

गति की गणना करने का एक सामान्य तरीका है मीटर प्रति सेकंड (मी/सेकंड) में व्यक्त की जाने वाली दूरी को समय के अनुसार विभाजित करना। यह सूत्र है:

गति = दूरी / समय

उदाहरण के लिए, यदि एक कार 100 मीटर की दूरी तय करती है और 20 सेकंड में ऐसा करती है, तो इसकी गति 100 मीटर / 20 सेकंड = 5 मीटर प्रति सेकंड होगी।

हालाँकि, गति को अक्सर किलोमीटर प्रति घंटा (किमी / घंटा) या मील प्रति घंटा (मील / घंटा) जैसे अन्य इकाइयों में भी व्यक्त किया जा सकता है, विशेष रूप से सड़क यातायात के संदर्भ में। ये इकाइयाँ 1 घंटे में समय के बजाय


In [21]:
# With Template

# Format message with the command-r-plus chat template
messages = [{"role": "user", "content": "What is the definition of velocity?"}]
input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")

## <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>What is the definition of velocity?<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>

input_ids = input_ids.to(device)

gen_tokens = model.generate(
    input_ids, 
    max_new_tokens=500, 
    do_sample=True, 
    temperature=0.3,
    )

gen_text = tokenizer.decode(gen_tokens[0])
print(gen_text)

<BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>What is the definition of velocity?<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>Velocity is the rate of change of velocity with respect to time. It is a vector quantity, which means that it has both magnitude and direction. The SI unit of velocity is meters per second (m/s).<|END_OF_TURN_TOKEN|>


In [20]:
# Without Template

message = "गति कैसे परिभाषित की जाती है?"

input_ids = tokenizer.encode(message, return_tensors="pt")

input_ids = input_ids.to(device)

gen_tokens = model.generate(
    input_ids, 
    max_new_tokens=500, 
    do_sample=True, 
    temperature=0.3,
    )

gen_text = tokenizer.decode(gen_tokens[0])

print(gen_text)

<BOS_TOKEN>गति कैसे परिभाषित की जाती है? »
क्या होता है जब हम किसी चीज़ को देखते हैं? »
क्या होता है जब हम किसी चीज़ को सुनते हैं? »
क्या होता है जब हम किसी चीज़ को छूते हैं? »
क्या होता है जब हम किसी चीज़ को सूँघते हैं? »
क्या होता है जब हम किसी चीज़ को चखकर खाते हैं? »
क्या होता है जब हम किसी चीज़ को स्पर्श करते हैं? »
क्या होता है जब हम किसी चीज़ को सुनाते हैं? »
क्या होता है जब हम किसी चीज़ को छूते हैं? »
क्या होता है जब हम किसी चीज़ को सूँघते हैं? »
क्या होता है जब हम किसी चीज़ को चखकर खाते हैं? »
क्या होता है जब हम किसी चीज़ को स्पर्श करते हैं? »
क्या होता है जब हम किसी चीज़ को सुनाते हैं? »
क्या होता है जब हम किसी चीज़ को छूते हैं? »
क्या होता है जब हम किसी चीज़ को सूँघते हैं? »
क्या होता है जब हम किसी चीज़ को चखकर खाते हैं? »
क्या होता है जब


In [22]:
# Without Template

message = "What is the definition of velocity?"

input_ids = tokenizer.encode(message, return_tensors="pt")

input_ids = input_ids.to(device)

gen_tokens = model.generate(
    input_ids, 
    max_new_tokens=500, 
    do_sample=True, 
    temperature=0.3,
    )

gen_text = tokenizer.decode(gen_tokens[0])

print(gen_text)

<BOS_TOKEN>What is the definition of velocity?
Velocity is the rate of change of position of an object with respect to time.
What is the definition of acceleration?
Acceleration is the rate of change of velocity with respect to time.
What is the definition of a vector?
A vector is a quantity that has both magnitude and direction.
What is the definition of a scalar?
A scalar is a quantity that has only magnitude.
What is the definition of a unit vector?
A unit vector is a vector with a magnitude of 1.
What is the definition of a force?
A force is a push or pull on an object.
What is the definition of a torque?
A torque is a force that acts at a distance from a point of rotation.
What is the definition of a moment?
A moment is the product of a force and a distance.
What is the definition of a moment of inertia?
A moment of inertia is the product of a moment and a mass.
What is the definition of a moment of force?
A moment of force is the product of a moment and a torque.
What is the defi